In [11]:
import pandas as pd
import numpy as np

In [12]:
data_path = 'csv/nepsent-1.csv'

df = pd.read_csv(data_path)

In [13]:
df = df[['tag','word']]

In [14]:
#Convert CSV to nltk word ,tag form
corpus= []
temp = []
for word,tag in zip(df['word'],df['tag']):
    if word != '.':
#         word = stemmer.stem(str(word))
        temp.append((word,tag))
    else:
        corpus.append(temp)
        temp=[]
print("Sample document: ",corpus[0])

Sample document:  [('महिला', 'NN'), ('समालोचक', 'NN'), ('र', 'CC'), ('नेपाली', 'JX'), ('समालोचना', 'NN'), ('।', 'YF')]


In [15]:
#Seperate word and tags into attributes and labels
sentences = []
sentence_tags = []

for sentence in corpus:
    if len(sentence)<200:
        x=[]
        y=[]
        for word in sentence:
            x.append(word[0])
            y.append(word[1])
        if len(x) > 0:
            sentences.append(x)
            sentence_tags.append(y)
print("Sample sentence: ",sentences[10])
print("Sample sentence tags: ",sentence_tags[10])

Sample sentence:  ['समालोचनालेखन', 'का', 'लागि', 'पर्याप्त', 'अध्ययन', 'को', 'आवश्यकता', 'मात्र', 'ले', 'पुग्दैन', ',', 'विवेचना', 'र', 'विश्लेषण', 'गर्ने', 'क्षमता', 'को', 'पनि', 'आवश्यकता', 'पर्दछ', '।', '।']
Sample sentence tags:  ['NN', 'IKO', 'II', 'JX', 'NN', 'IKM', 'NN', 'TT', 'IE', 'VVYN1', 'YM', 'NN', 'CC', 'NN', 'VN', 'NN', 'IKM', 'TT', 'NN', 'VVYN1', 'YF', 'YF']


In [16]:
#Convert labels to numbers
labels = set()
for sentence in sentence_tags:
    for tag in sentence:
        labels.add(tag)
        
tag2index = {t: i + 1 for i, t in enumerate(list(labels))}
print("Total number of tags: ",labels)

Total number of tags:  {'VDM', 'IKX', 'VS', 'VOYX2', 'VOYN1', 'UU', 'PRF', 'QQ', 'DDX', 'VVYX2', 'MOM', 'PRFKX', 'CSB', 'PMXKM', 'VCN', 'YB', 'FO', 'DGF', 'VVMX2', 'PTH', 'VCM', 'CC', 'DJM', 'VR', 'FB', 'YM', 'RK', 'FF', 'VOMX2', 'VVTN1', 'VI', 'PXR', 'RD', 'DDM', 'PRFKF', 'JT', 'IKM', 'VVMX1', 'DGO', 'VVTN1F', 'NN', 'VDF', 'PMX', 'PRFKM', 'VVYN1F', 'DJX', 'DDO', 'IA', 'FZ', 'JO', 'FU', 'YQ', 'MM', 'PXH', 'CSA', 'VOMX1', 'RR', 'PTNKX', 'MOX', 'JX', 'JF', 'VVYN1', 'DKM', 'PTNKM', 'MLO', 'PMXKF', 'VDX', 'VVTX2', 'IKO', 'IE', 'PMXKO', 'DKX', 'VE', 'JM', 'DGX', 'PTN', 'DKF', 'PTMKF', 'FS', 'PTMKO', 'PTM', 'IKF', 'PTMKX', 'YF', 'MLF', 'DKO', 'DGM', 'PRFKO', 'II', 'NP', 'PMXKX', 'MLX', 'VQ', 'IH', 'VN', 'VVYM1F', 'PTMKM', 'VCH', 'TT', 'RJ', 'VDO'}


In [17]:
def tagsent2int(sent_tag):
    
    return [tag2index[tag] for tag in sent_tag]

sentence_tags = list(map(tagsent2int,sentence_tags))

In [18]:
#Convert label nums to categorical
def to_categorical(sequences,categories):
    cat_sequences = []
    for s in sequences:
        cats = []
        for item in s:
            cats.append(np.zeros(categories))
        cat_sequences.append(cats)
    return np.array(cat_sequences)

In [19]:
#Preprocess text to numbers
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
t = Tokenizer(lower=False, oov_token='-PAD-')
t.fit_on_texts(sentences)
vocab_size = len(t.word_index) + 1

encoded_docs = t.texts_to_sequences(sentences)

print("Sample encoded doc 0 : ",encoded_docs[:2]," Shape: " ,np.asarray(encoded_docs).shape)

max_length = len(max(sentences,key=len))
padded_docs = pad_sequences(encoded_docs, maxlen=max_length, padding='post')


Using TensorFlow backend.


Sample encoded doc 0 :  [[129, 811, 8, 27, 127, 2], [2530, 1788, 2]]  Shape:  (16633,)
